Trying various models for sentiment classification

In [1]:
%pip install transformers

  Using cached pyyaml-6.0.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.0 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 20.7 MB/s  0:00:00
Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
Using cached fsspec-2025.9.0-py3-none-any.wh

In [2]:
%pip install --upgrade protobuf tiktoken sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import pipeline

# RoBERTA, 7 labels, lightweight, fast-ish
emotion_model_1 = pipeline("text-classification", 
                           model="j-hartmann/emotion-english-distilroberta-base", 
                           return_all_scores=True) 
# T5, 6 labels, larger model, slower
emotion_model_2 = pipeline("text-classification", 
                           model="mrm8488/t5-base-finetuned-emotion", 
                           return_all_scores=True)
# DistilBERT, 6 labels, lightweight, fast, less accurate / more coarse
emotion_model_3 = pipeline("text-classification", 
                           model="bhadresh-savani/distilbert-base-uncased-emotion", 
                           return_all_scores=True)
# BERT, 6 labels, larger model, slower, need GPU
emotion_model_4 = pipeline("text-classification", 
                           model="nateraw/bert-base-uncased-emotion", 
                           return_all_scores=True)
# RoBERTA, 28 labels, larger model, slowest
emotion_model_5 = pipeline("text-classification",
                           model="SamLowe/roberta-base-go_emotions", 
                           return_all_scores=True)

Device set to use cpu
/Users/nandinibohra/Desktop/VSCodeFiles/CatGPT/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mrm8488/t5-base-finetuned-emotion and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [9]:
def get_emotion_predictions(text):
    results = {
        "j-hartmann/emotion-english-distilroberta-base": emotion_model_1(text),
        "mrm8488/t5-base-finetuned-emotion": emotion_model_2(text),
        "bhadresh-savani/distilbert-base-uncased-emotion": emotion_model_3(text),
        "nateraw/bert-base-uncased-emotion": emotion_model_4(text),
        "SamLowe/roberta-base-go_emotions": emotion_model_5(text)
    }
    return results

texts = ["I failed my exam today.",                   # Sad 
         "I am so excited about my new job!",         # Joy, surpise
         "My friends are throwing me a party!",       # Joy, surprise
         "I threw up on my shoes.",                   # Disgust, sad
         "I'm worried my cat is sick.",               # Fear
         "My best friend is getting married.",        # Joy
         "The bed is warm.",                           # Neutral, joy
         "Hi",                                         # Neutral 
         "My boyfriend cheated on me.",                # Anger, sadness
         "My dog peed on the couch."                 # Disgust, anger
    ]

for text in texts:
    print(f"Input Text: {text}")
    predictions = get_emotion_predictions(text)
    for model_name, result in predictions.items():
        print(f"Model: {model_name}")
        for score in result[0]:
            print(f"  {score['label']}: {score['score']:.4f}")
        print()
    print("--------------------------------------------------\n")
    predictions = get_emotion_predictions(text)

Input Text: I failed my exam today.
Model: j-hartmann/emotion-english-distilroberta-base
  anger: 0.0196
  disgust: 0.0205
  fear: 0.0046
  joy: 0.0019
  neutral: 0.0032
  sadness: 0.9487
  surprise: 0.0015

Model: mrm8488/t5-base-finetuned-emotion
  LABEL_0: 0.6917
  LABEL_1: 0.3083

Model: bhadresh-savani/distilbert-base-uncased-emotion
  sadness: 0.1444
  joy: 0.0256
  love: 0.0013
  anger: 0.3166
  fear: 0.5086
  surprise: 0.0034

Model: nateraw/bert-base-uncased-emotion
  sadness: 0.6891
  joy: 0.0081
  love: 0.0021
  anger: 0.1591
  fear: 0.1373
  surprise: 0.0042

Model: SamLowe/roberta-base-go_emotions
  admiration: 0.0043
  amusement: 0.0017
  anger: 0.0043
  annoyance: 0.0486
  approval: 0.0109
  caring: 0.0024
  confusion: 0.0031
  curiosity: 0.0017
  desire: 0.0033
  disappointment: 0.7604
  disapproval: 0.0396
  disgust: 0.0046
  embarrassment: 0.0138
  excitement: 0.0026
  fear: 0.0030
  gratitude: 0.0014
  grief: 0.0041
  joy: 0.0053
  love: 0.0055
  nervousness: 0.0077
